Installation of necessary packages (and creation of a conda env):
```
./install_main.sh 
```

Process results obtained via running command:
```
bash -i run_mnli_main.sh
```

In [1]:
SEED_QTY=3
OUTPUT_PREF='results_mnli/output_res_'

In [2]:
#sync_step_arr = [1, 2, 4, 8, 16]
sync_step_arr_accum = [1, 2, 4, 8, 16, 32, 64, 128, 256]
sync_step_arr_nosync = [1, 2, 4, 8, 16, 32, 64, 128, 256]

res_keys = ['1gpu'] + \
        [f'accum_steps_{k}' for k in sync_step_arr_accum] + \
        [f'nosync_steps_{k}' for k in sync_step_arr_nosync]

In [3]:
import json

res_dict = {k : [] for k in res_keys}

for train_qty in [4000, 40000]:
    for rk in res_keys:
        one_res = {}
        
        for seed in range(SEED_QTY):
            tmp = json.load(open(f'{OUTPUT_PREF}{train_qty}_{rk}/{seed}/all_results.json'))
            if seed == 0:
                for mk in tmp.keys():
                    one_res[mk] = []

            for mk in tmp.keys():
                one_res[mk].append(tmp[mk])
                 
        res_dict[rk].append(one_res)
    

In [4]:
res_dict['1gpu']

[{'eval_accuracy': [0.7508136696501221,
   0.7376932465419039,
   0.7201993490642798],
  'train_time': [74.37206363677979, 73.83928155899048, 74.55089211463928]},
 {'eval_accuracy': [0.3295362082994304,
   0.8177379983726607,
   0.8218063466232709],
  'train_time': [730.9971737861633, 730.1912815570831, 727.5648543834686]}]

In [5]:
tmp

{'eval_accuracy': 0.7979910714285714, 'train_time': 190.1690673828125}

In [6]:
import numpy as np

res_dict_str = {k : [] for k in res_keys}

for cidx, train_qty in enumerate([4000, 40000]):
    for rk in res_keys:
        speed_up = round(np.mean(res_dict['1gpu'][cidx]['train_time']) / np.mean(res_dict[rk][cidx]['train_time']), 1)
        acc = str(round(np.mean(res_dict[rk][cidx]['eval_accuracy']), 2)) +'/' +str(round(np.max(res_dict[rk][cidx]['eval_accuracy']), 2))
        #print(rk, speed_up, f1)
        if rk != '1gpu':
            res_dict_str[rk].append(f'{acc} {speed_up}x')
        else:
            res_dict_str[rk].append(f'{acc}')  
            
            
res_dict_str

{'1gpu': ['0.74/0.75', '0.66/0.82'],
 'accum_steps_1': ['0.62/0.64 1.4x', '0.81/0.81 1.5x'],
 'accum_steps_2': ['0.35/0.37 4.2x', '0.78/0.78 4.4x'],
 'accum_steps_4': ['0.33/0.36 10.6x', '0.6/0.66 11.9x'],
 'accum_steps_8': ['0.33/0.35 22.9x', '0.35/0.37 28.2x'],
 'accum_steps_16': ['0.33/0.35 47.7x', '0.33/0.35 63.6x'],
 'accum_steps_32': ['0.33/0.35 59.4x', '0.33/0.35 117.7x'],
 'accum_steps_64': ['0.33/0.35 63.9x', '0.33/0.35 224.6x'],
 'accum_steps_128': ['0.33/0.35 63.7x', '0.33/0.35 310.5x'],
 'accum_steps_256': ['0.33/0.35 64.7x', '0.33/0.35 369.2x'],
 'nosync_steps_1': ['0.54/0.55 1.2x', '0.8/0.8 1.2x'],
 'nosync_steps_2': ['0.53/0.56 1.7x', '0.8/0.81 1.8x'],
 'nosync_steps_4': ['0.53/0.54 2.3x', '0.8/0.8 2.4x'],
 'nosync_steps_8': ['0.54/0.57 2.8x', '0.8/0.8 3.0x'],
 'nosync_steps_16': ['0.52/0.53 3.1x', '0.8/0.8 3.3x'],
 'nosync_steps_32': ['0.51/0.52 3.3x', '0.8/0.8 3.6x'],
 'nosync_steps_64': ['0.51/0.52 3.5x', '0.8/0.8 3.7x'],
 'nosync_steps_128': ['0.51/0.52 3.5x', '0.8/0

In [7]:
import pandas as pd
df=pd.DataFrame.from_dict(res_dict_str)

In [8]:
df

,1gpu,accum_steps_1,accum_steps_2,accum_steps_4,accum_steps_8,accum_steps_16,accum_steps_32,accum_steps_64,accum_steps_128,accum_steps_256,nosync_steps_1,nosync_steps_2,nosync_steps_4,nosync_steps_8,nosync_steps_16,nosync_steps_32,nosync_steps_64,nosync_steps_128,nosync_steps_256
0,0.74/0.75,0.62/0.64 1.4x,0.35/0.37 4.2x,0.33/0.36 10.6x,0.33/0.35 22.9x,0.33/0.35 47.7x,0.33/0.35 59.4x,0.33/0.35 63.9x,0.33/0.35 63.7x,0.33/0.35 64.7x,0.54/0.55 1.2x,0.53/0.56 1.7x,0.53/0.54 2.3x,0.54/0.57 2.8x,0.52/0.53 3.1x,0.51/0.52 3.3x,0.51/0.52 3.5x,0.51/0.52 3.5x,0.51/0.52 3.5x
1,0.66/0.82,0.81/0.81 1.5x,0.78/0.78 4.4x,0.6/0.66 11.9x,0.35/0.37 28.2x,0.33/0.35 63.6x,0.33/0.35 117.7x,0.33/0.35 224.6x,0.33/0.35 310.5x,0.33/0.35 369.2x,0.8/0.8 1.2x,0.8/0.81 1.8x,0.8/0.8 2.4x,0.8/0.8 3.0x,0.8/0.8 3.3x,0.8/0.8 3.6x,0.8/0.8 3.7x,0.8/0.8 3.8x,0.8/0.8 3.8x


In [9]:
#df.to_csv('results.tsv')

In [10]:
#!cat results.tsv